In [1]:
import os
from dotenv import load_dotenv
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime

load_dotenv(override=True)

DATA_PATH = os.getenv('DATA_PATH')
print(DATA_PATH)

# Choose subontology (CCO, MFO or BPO)
SO = 'CCO'

2023-05-17 17:38:14.685375: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 17:38:15.113515: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


/mnt/e/ML/cafa-5-protein-function-prediction


## Reading fasta, obo and tsv files

In [2]:
from Bio import SeqIO

sequences = [rec.seq for rec in SeqIO.parse(os.path.join(DATA_PATH, "Test (Targets)/testsuperset.fasta"),"fasta")]
ids = [rec.id for rec in SeqIO.parse(os.path.join(DATA_PATH, "Test (Targets)/testsuperset.fasta"),"fasta")]

In [3]:
print("There are {} sequences in the dataset.".format(len(sequences)))

There are 141865 sequences in the dataset.


In [4]:
import networkx
import obonet

# Read the taxrank ontology
url = os.path.join(DATA_PATH, "Train/go-basic.obo")
graph = obonet.read_obo(url)



## Label encoding

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer
import pickle 

# Hydrate the serialized objects.
with open(os.path.join(DATA_PATH,'MLB_'+SO+'.pkl'), 'rb') as f:
    mlb = pickle.load(f)

print(len(mlb.classes_))

2957


## Amino acids encoding

In [6]:
aminos_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'X']

In [7]:
aa_dict = {'A': 1, 'B':24, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'O': 21, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'U': 22, 'V': 18, 'W': 19, 'Y': 20, 'X':30, 'Z':23}

## Build Dataset

In [8]:
maxLen = 35375

In [9]:


mapping = lambda x: aa_dict[x]
vectMapping = np.vectorize(mapping)

sequences=sequences[0:500]
ids=ids[0:500]



def generator(padding=True):
    for i,seq in enumerate(sequences):
        entryId = ids[i]
        
        arr = np.array(seq)
        mappedArr = vectMapping(arr)
        if padding:
            padWidth = maxLen - arr.size
            mappedArr = np.pad(mappedArr, (0, padWidth))
        yield entryId, mappedArr



In [10]:
g = generator()
test = next(g)
print("The first sample sequence: {}".format(test))

The first sample sequence: ('Q9CQV8', array([11, 17, 11, ...,  0,  0,  0]))


## Tensorflow Classification

In [11]:
import tensorflow as tf


dataset = tf.data.Dataset.from_generator(generator, output_signature=(tf.TensorSpec(shape=(),dtype=tf.dtypes.string),
         tf.TensorSpec(shape=(maxLen,), dtype=tf.int32)))
print(list(dataset.take(1)))


2023-05-17 17:38:23.481285: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-17 17:38:23.499353: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-17 17:38:23.499675: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-17 17:38:23.501085: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-17 17:38:23.501383: I tensorflow/compile

[(<tf.Tensor: shape=(), dtype=string, numpy=b'Q9CQV8'>, <tf.Tensor: shape=(35375,), dtype=int32, numpy=array([11, 17, 11, ...,  0,  0,  0], dtype=int32)>)]


r/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-17 17:38:24.002609: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-17 17:38:24.002929: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-17 17:38:24.002941: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1722] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2023-05-17 17:38:24.003252: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to r

In [12]:
CCOmodel = tf.keras.saving.load_model(os.path.join(DATA_PATH, "model_CCO_epoch_1_valAcc0.997"))

In [13]:

# probs= CCOmodel.predict(tf.expand_dims(list(dataset.take(64))[0][1], 0))
# prediction= [1 if p > 0.5 else 0 for p in probs[0]]
# probabilities= probs[probs>0.5]
# # classes = np.argwhere(prediction)
# print(mlb.inverse_transform(np.array([prediction])))
# print(probabilities)


dataset = dataset.batch(64)
tableData=[]

for entries, data in dataset:

    probs= CCOmodel.predict(data)

    for i,prob in enumerate(probs):
        prediction= [1 if p > 0.5 else 0 for p in prob]
        probabilities= prob[prob>0.5]
        entry = entries[i]
        GOs = mlb.inverse_transform(np.array([prediction]))

        for j,g in enumerate(GOs[0]):
            tableData.append([entry.numpy().decode("utf-8") , g, probabilities[j]])

        
results = pd.DataFrame(tableData, columns=['Entry ID', 'GO', 'Probability'])


2023-05-17 17:38:24.647303: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-17 17:38:25.155564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


2/2 [==============================] - 0s 118ms/step


In [ ]:
#TODO: Add inference for all 3 models inside the dataset loop